In [2]:
import os
import pandas as pd
os.getcwd()


from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import TargetEncoder, LabelEncoder

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import pandas as pd
import time
import pickle
from xgboost import XGBClassifier

import pandas as pd
import time
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split

NUMERIC_COLS = [
    "TP_FAIXA_ETARIA",
    "TP_ESTADO_CIVIL",
    "TP_COR_RACA",
    "TP_NACIONALIDADE",
    "TP_ST_CONCLUSAO",
    "TP_ANO_CONCLUIU",
    "IN_TREINEIRO",
    "CO_MUNICIPIO_PROVA",
    "CO_UF_PROVA",
    "TP_PRESENCA_CN",
    "TP_PRESENCA_CH",
    "TP_PRESENCA_LC",
    "TP_PRESENCA_MT",
    "NU_NOTA_CN",
    "NU_NOTA_CH",
    "NU_NOTA_LC",
    "NU_NOTA_MT",
    "TP_LINGUA",
    "NU_NOTA_REDACAO",
    "NU_NOTA_COMP1",
    "NU_NOTA_COMP2",
    "NU_NOTA_COMP3",
    "NU_NOTA_COMP4",
    "NU_NOTA_COMP5"
]

CATEGORICAL_COLS = [
    "TP_SEXO",
    "Q001",
    "Q002",
    "Q003",
    "Q004",
    "Q007",
    "Q008",
    "Q009",
    "Q010",
    "Q011",
    "Q012",
    "Q013",
    "Q014",
    "Q015",
    "Q016",
    "Q017",
    "Q018",
    "Q019",
    "Q020",
    "Q021",
    "Q022",
    "Q023",
    "Q024",
    "Q025",
    "NO_MUNICIPIO_PROVA",
    "TP_STATUS_REDACAO",
    "SG_UF_PROVA",
    "faixa_renda_familiar"
]


def generate_pipeline():
    numeric_transformer = Pipeline(steps=[("scaler", MinMaxScaler())])

    categorical_transformer = Pipeline(steps=[("encoder", OrdinalEncoder())])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, NUMERIC_COLS),
            ("cat", categorical_transformer, CATEGORICAL_COLS),
        ]
    )

    return Pipeline(steps=[("preprocessor", preprocessor)])


In [3]:
df_microdados = pd.read_parquet('../../tcc-results/data-results/data_prepared_enem_2022.parquet')

In [4]:

df_microdados = df_microdados.loc[df_microdados["TP_ESCOLA"] != 1]
print(df_microdados.shape)

X = df_microdados.drop('TP_ESCOLA', axis=1)
y = df_microdados['TP_ESCOLA']

# replace target values [2, 3] to [0, 1]
y = y.replace({2: 0, 3: 1})

(1317560, 54)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBClassifier(n_estimators=400, objective = 'multi:softmax', num_class = 2, eval_metric='logloss', n_jobs=-1)

k_folds = 5

pipe = generate_pipeline()
            
X_train_method = pipe.fit_transform(X_train)
X_test_transformed = pipe.transform(X_test)

kf = KFold(n_splits=k_folds, shuffle=True)
cv_results = cross_val_score(model, X_train_method, y_train, cv=kf)

# get training time
start_time = time.time()
model.fit(X_train_method, y_train)
end_time = time.time()

# save model
with open(f'/home/cristiano/ufpr/tcc/binaria/baseline/modelos/binary_baseline_feature_selection.pkl', 'wb') as file:
    pickle.dump(model, file)

training_time = end_time - start_time
print("Tempo de treinamento:", training_time)

y_predicted = model.predict(X_test_transformed)

# get classification metrics
test_accuracy = accuracy_score(y_test, y_predicted)
test_f1_score = f1_score(y_test, y_predicted)
test_precision = precision_score(y_test, y_predicted)
test_recall = recall_score(y_test, y_predicted)

print("Accuracy médio na validação cruzada:", np.mean(cv_results))
print("Accuracy no conjunto de teste:", test_accuracy)

print("F1 Score no conjunto de teste:", test_f1_score)
print("Precision no conjunto de teste:", test_precision)

Tempo de treinamento: 24.872456073760986
Accuracy médio na validação cruzada: 0.8899129827824334
Accuracy no conjunto de teste: 0.8901795743647348
F1 Score no conjunto de teste: 0.6105061979299856
Precision no conjunto de teste: 0.7131402697858693
